In [ ]:
from glob import glob
from natsort import natsorted
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use('CMS')

In [ ]:
import qinj_analysis_helper as helper

helper.toSingleDataFramePerDirectory_newEventModel(
    path_to_dir = '../../ETROC-Data/cern_Jan2024/TID/*PlateTemp12C_DryAir*/',
    dir_name_pattern = 'ET2p01_W36_6_VRef_SCurve_TDC_*',
    output_dir = '../../ETROC-Data/cern_Jan2024/TID/0MRad_Data/',
    extra_str = '_p12c_0MRad',
    # save_to_csv = True,
    # debugging = True
)

In [ ]:
files = natsorted(glob('../../ETROC-Data/cern_Jan2024/TID/0MRad_Data/*feather'))
files

In [ ]:
data_dict = {
    'row': [],
    'col': [],
    'charge': [],
    'dac': [],
    'toa_mean': [],
    'toa_std': [],
    'tot_mean': [],
    'tot_std': [],
    'cal_mean': [],
    'cal_std': [],
}

for ifile in tqdm(files):
    fname = ifile.split('/')[-1].split('.')[0]

    # row = fname.split('_')[-5].split('R')[1]
    # col = fname.split('_')[-6].split('C')[1]
    # charge = fname.split('_')[-3]
    # dac = fname.split('_')[-1]

    row = fname.split('_')[-7].split('R')[1]
    col = fname.split('_')[-8].split('C')[1]
    charge = fname.split('_')[-5]
    dac = fname.split('_')[-3]

    # print(row, col, charge, dac)

    df = pd.read_feather(ifile)
    df = df.loc[(df['cal'] >= df['cal'].mode()[0]-2) & (df['cal'] <= df['cal'].mode()[0]+2)]

    data_dict['row'].append(int(row))
    data_dict['col'].append(int(col))
    data_dict['charge'].append(int(charge))
    data_dict['dac'].append(int(dac))

    data_dict['toa_mean'].append(df['toa'].mean())
    data_dict['toa_std'].append(df['toa'].std())
    data_dict['tot_mean'].append(df['tot'].mean())
    data_dict['tot_std'].append(df['tot'].std())
    data_dict['cal_mean'].append(df['cal'].mean())
    data_dict['cal_std'].append(df['cal'].std())

In [ ]:
cut_df = pd.DataFrame(data=data_dict)

In [ ]:
final_df = cut_df.loc[(cut_df['toa_std'] <= 5)]

In [ ]:
final_df.to_feather('ET2p01_W36_6_TDC_summary_p12C_0MRad.feather')

In [ ]:
# final_df = pd.read_feather('./ET2p01_Bar4_TDC_summary_p22C.feather')
# final_df = pd.read_feather('./ET2p01_Bar4_TDC_summary_m25C.feather')
final_df = pd.read_feather('./ET2p01_W36_6_TDC_summary_p12C_0MRad.feather')
final_df

In [ ]:
final_df.loc[(final_df['row'] == 1)]['col'].unique()
# final_df.loc[(final_df['col'] == 2)]['row'].unique()

# (0,2), (0,8), (0, 14)
# (1,2), (1,8), (1, 14)
# (3,2), (3,8), (3, 14)
# (12,2), (12,8), (12, 14)

In [ ]:
def round_down_to_nearest_hundred(number):
    rounded_number = (number // 100) * 100
    return rounded_number

def round_down_to_nearest_ten(number):
    rounded_number = (number // 10) * 10
    return rounded_number

In [ ]:
def make_moneyplot(
        input_df: pd.DataFrame,
        pixel: list[set],
        value_name: str,
        board_name: str,
        title_tag: str,
        temperature: str,
        ylabel: str,
    ):

    sub_df = input_df.loc[(input_df['row'] == pixel[0]) & (input_df['col'] == pixel[1])]

    from cycler import cycler
    custom_cycler = (cycler(color=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02'])
                    + cycler(marker=['o', 'X', '^', 'D', 's', '*']))

    fig, ax = plt.subplots(figsize=(16, 9))
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    ax.set_prop_cycle(custom_cycler)
    ax.grid()

    for icharge in input_df['charge'].unique():
        tmp_df = sub_df.loc[sub_df['charge'] == icharge]
        plt.plot(tmp_df['dac'], tmp_df[value_name], label=f"{icharge} fC")

    ax.set_title(f"{board_name} Pixel ({pixel[0]},{pixel[1]}) {title_tag}, T: {temperature}", size=22, loc="right")
    ax.set_xlim(round_down_to_nearest_ten(tmp_df['dac'].min())-10, None)
    # ax.tick_params(axis='x', which='major', pad=7)
    # ax.tick_params(axis='y', which='major', pad=8)
    # ax.set_ylim(-0.2, 8)
    ax.legend(loc='upper right')
    ax.set_xlabel('DAC value [LSB]')
    ax.set_ylabel(ylabel)

In [ ]:
pixels = [(0,2), (0,8), (0,14), (3,2), (3,8), (3,14), (12,2), (12,8), (12,14)]

for ipix in pixels:
    make_moneyplot(final_df, ipix, value_name='toa_std', board_name='ET2.01 Bar4', title_tag='QInj Money plot', temperature='-25C', ylabel='TOA Std [LSB]')

In [ ]:
pixels = [(2,2), (2,8), (2,14), (8,2), (8,8), (8,14), (14,2), (14,8), (14,14)]
for ipix in pixels:
    make_moneyplot(final_df, ipix, value_name='toa_std', board_name='ET2.01 Bar4', title_tag='QInj Money plot',temperature='-25C', ylabel='TOA Std [LSB]')

In [ ]:
pixels = [(0,2), (0,8), (0,14), (1,2), (1,8), (1,14), (3,2), (3,8), (3,14), (12,2), (12,8), (12,14)]

for ipix in pixels:
    make_moneyplot(final_df, ipix, value_name='toa_mean', board_name='ET2.01 W36-6', title_tag='QInj Money plot 200MRad',temperature='-25C', ylabel='TOA Mean [LSB]')